# Connect G-Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
default_dir = "/content/drive/MyDrive/..."
os.chdir(default_dir)

In [ ]:
!ls

# Import Libraries

In [ ]:
import warnings
warnings.simplefilter(action="ignore")

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

from sklearn.preprocessing import RobustScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.ensemble import (
    GradientBoostingClassifier,
    RandomForestClassifier
)

from sklearn.metrics import (
    accuracy_score,
    roc_auc_score,
    roc_curve,
    classification_report
)

from sklearn.model_selection import (
    KFold,
    train_test_split,
    GridSearchCV,
    cross_val_score
)

import pickle

# 1.&nbsp;Load Dataset

In [ ]:
df = pd.read_csv("diabetes.csv")

In [ ]:
df.head()

## 1.1 Pima Indians Diabetes Database

This dataset is originally from the National Institute of Diabetes and Digestive and Kidney Diseases. The objective of the dataset is to diagnostically predict whether or not a patient has diabetes, based on certain diagnostic measurements included in the dataset. Several constraints were placed on the selection of these instances from a larger database. In particular, **all patients here are females at least 21 years old of Pima Indian heritage**.

The datasets consists of several medical predictor variables and one target variable, **Outcome**. Predictor variables includes the number of pregnancies the patient has had, their BMI, insulin level, age, and so on.

We build a **machine learning model** to accurately predict whether or not the patients in the dataset have **diabetes or not.**

- **Pregnancies**: Number of times pregnant
- **Glucose**: Plasma glucose concentration a 2 hours in an oral glucose tolerance test
- **BloodPressure**: Diastolic blood pressure (mm Hg)
- **SkinThickness**: Triceps skin fold thickness (mm)
- **Insulin**: 2-Hour serum insulin (mu U/ml)
- **BMI**: Body mass index (weight in kg/(height in m)^2)
- **DiabetesPedigreeFunction**: Diabetes pedigree function
- **Age**: Age (years)
- **Outcome**: Class variable (0 or 1) 268 of 768 are 1, the others are 0

## 1.2 General Information on Variables

### a. Glucose Tolerance Test
It is a blood test that involves taking multiple blood samples over time, usually 2 hours.It used to diagnose diabetes. The results can be classified as normal, impaired, or abnormal.
* **Normal Results for Diabetes ->** Two-hour glucose level less than 140 mg/dL

* **Impaired Results for Diabetes ->** Two-hour glucose level 140 to 200 mg/dL

* **Abnormal (Diagnostic) Results for Diabetes ->** Two-hour glucose level greater than 200 mg/dL



### b. BloodPressure
The diastolic reading, or the bottom number, is the pressure in the arteries when the heart rests between beats. This is the time when the heart fills with blood and gets oxygen. A normal diastolic blood pressure is lower than 80. A reading of 90 or higher means you have high blood pressure.

* **Normal**: Systolic below 120 and diastolic below 80
* **Elevated**: Systolic 120–129 and diastolic under 80
* **Hypertension stage 1**: Systolic 130–139 and diastolic 80–89
* **Hypertension stage 2**: Systolic 140-plus and diastolic 90 or more
* **Hypertensive crisis**: Systolic higher than 180 and diastolic above 120.

### c. BMI

The standard weight status categories associated with BMI ranges for adults are shown in the following table.

* Below 18.5 -> **Underweight**
* 18.5 – 24.9 -> **Normal or Healthy Weight**
* 25.0 – 29.9 -> **Overweight**
* 30.0 and Above -> **Obese**

### d. Triceps Skinfolds
For adults, the standard normal values for triceps skinfolds are:
* 18.0mm (women)

# 2.&nbsp;Exploratory Data Analysis

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
# Getting various summary statistics
# There is notably a large difference between 99% and max values of predictors
# “Insulin”, ”SkinThickness”, ”DiabetesPedigreeFunction”
# There are extreme values-Outliers in our data set

# See BMI Min: 0
df.describe(
    percentiles=[0.05, 0.25, 0.50, 0.75, 0.90, 0.95, 0.99]
).T

In [ ]:
# Target Variable: Categorical
df['Outcome'].unique()

In [ ]:
df['Outcome'].value_counts()

In [ ]:
df['Outcome'].value_counts(normalize=True)

# 3.&nbsp;Data Visualization

In [ ]:
plt.figure(figsize=(8, 6))
sns.heatmap(
    df.corr(),
    cmap='Blues',
    annot=True
);

In [ ]:
df.nlargest(10, 'BloodPressure')

In [ ]:
# df.corr()

In [ ]:
k = 10
k_largest_corr = df.corr().nlargest(k, 'Outcome')
k_largest_feats = k_largest_corr['Outcome'].index
list(k_largest_feats)

In [ ]:
# Outcome correlation matrix

k = 9 # number of variables for heatmap
cols = df.corr().nlargest(k, 'Outcome')['Outcome'].index
corr_mat = df[cols].corr()

# Visualize
plt.figure(figsize=(10, 5))
sns.heatmap(
    corr_mat, cmap='viridis', annot=True,
);

In [ ]:
# df.loc[df.Pregnancies==12]

In [ ]:
# see how the data is distributed.
df.hist(figsize=(20,20));

In [ ]:
df['Age'].mean(), df['Age'].median()

In [ ]:
for col in df.columns:
    if col != "Outcome":
        sns.catplot(
            data=df, x="Outcome",
            y=col, hue="Outcome")
        plt.grid()

# 4.&nbsp;Data Preprocessing

In [ ]:
# Observation units for variables with a minimum value of zero are NaN,
# except for the pregnancy variable.
df.describe(
    percentiles=[0.05, 0.25, 0.50, 0.75, 0.90, 0.95, 0.99]
).T

## Handling Missing Values: Imputation

In [ ]:
# NaN values of 0 for Glucose, Blood Pressure, Skin Thickness, Insulin, BMI
# We can write Nan instead of 0

cols = ["Glucose", "BloodPressure", "SkinThickness", "Insulin", "BMI"]
for col in cols:
    df[col].replace(0, np.NaN, inplace=True)

In [ ]:
# now we can see missing values
df.isnull().sum()

In [ ]:
df["Outcome"] == 0

In [ ]:
# We can fill in NaN values with a median
# according to the target value

cols = [
    "Glucose",
    "BloodPressure",
    "SkinThickness",
    "Insulin",
    "BMI"
]

mask_label_zero = (df["Outcome"] == 0)
mask_label_one = (df["Outcome"] == 1)

for col in cols:

    mask_col_null = df[col].isnull()
    col_median_zero = df[mask_label_zero][col].median()
    col_median_one = df[mask_label_one][col].median()

    df.loc[(mask_label_zero & mask_col_null), col] = col_median_zero
    df.loc[(mask_label_one & mask_col_null), col] = col_median_one

In [ ]:
df.isnull().sum()

In [ ]:
# df[(df['Pregnancies'] == 1)].any(axis=0)
# df[(df['Pregnancies'] == 1)].any(axis=None)
# (df['Pregnancies'] == 1).all(axis=None)

## Outlier Handling

In [ ]:
def outlier_thresholds(
        df, feature,
        quantile_lower=0.25,
        quantile_upper=0.75):

    Q1 = df[feature].quantile(quantile_lower)
    Q3 = df[feature].quantile(quantile_upper)
    IQR = Q3 - Q1

    lower_limit = Q1 - 1.5 * IQR
    upper_limit = Q3 + 1.5 * IQR

    return lower_limit, upper_limit

In [ ]:
def has_outliers(
        df, feature,
        quantile_lower=0.25,
        quantile_upper=0.75):
    """
    Args:
        df (pd.DataFrame): DataFrame containing feature
        feature (str): feature name to be checked

    Return:
        bool: Is outlier(s) exist in given feature in the DataFrame
    """
    low_lim, up_lim = outlier_thresholds(
        df, feature, quantile_lower, quantile_upper)
    exist_lower_outliers = (df[feature] < low_lim).any(axis=None)
    exist_upper_outliers = (df[feature] > up_lim).any(axis=None)

    return (exist_lower_outliers or exist_upper_outliers)

In [ ]:
for feat in df.columns:
    exist_outliers = has_outliers(
        df, feat, quantile_lower=0.1, quantile_upper=0.9)
    if exist_outliers:
        print(f"Outliers exist in {feat}!")

In [ ]:
feats_with_outliers = []

for feat in df.columns:
    exist_outliers = has_outliers(
        df, feat, quantile_lower=0.1, quantile_upper=0.9)
    if exist_outliers:
        feats_with_outliers.append(feat)

In [ ]:
feats_with_outliers

In [ ]:
def replace_with_thresholds(df, numerical_feats):
    for feat in numerical_feats:
        low_limit, up_limit = outlier_thresholds(df, feat)

        mask_lower = (df[feat] < low_limit)
        mask_upper = (df[feat] > up_limit)

        df.loc[mask_lower, feat] = low_limit
        df.loc[mask_upper, feat] = up_limit

In [ ]:
replace_with_thresholds(df,feats_with_outliers)

In [ ]:
df.describe(
    percentiles=[0.05, 0.25, 0.50, 0.75, 0.90, 0.95, 0.99]).T

# 5.&nbsp;Feature Engineering

See 1.1 & 1.2

## Feature Categorization

In [ ]:
max(df['Glucose'])

In [ ]:
df['New_Glucose_Class'] = pd.cut(
    x=df['Glucose'],
    bins=[0, 139, 200],
    labels=["Normal", "Pre-Diabetes"]
)

In [ ]:
df['New_Glucose_Class'].value_counts(normalize=True)

In [ ]:
df['New_BMI_Range'] = pd.cut(
    x=df['BMI'],
    bins=[0, 18.5, 24.9, 29.9, 100],
    labels=["Underweight", "Healthy", "Overweight", "Obese"]
)

df['New_BMI_Range'].value_counts(normalize=True)

In [ ]:
df['New_BloodPressure'] = pd.cut(
    x=df['BloodPressure'],
    bins=[0, 79, 89, 123],
    labels=["Normal", "HS1", "HS2"]
)

df['New_BloodPressure'].value_counts(normalize=True)

In [ ]:
df['New_SkinThickness'] = (
    df['SkinThickness']
    .apply(lambda x: 1 if x <= 18.0 else 0)
)

In [ ]:
df.head()

## One-Hot Encoding

In [ ]:
def one_hot_encoder(
        df, categorical_feats,
        nan_as_category=False):

    original_columns = list(df.columns)

    df = pd.get_dummies(
        df,
        columns=categorical_feats,
        dummy_na=nan_as_category,
        drop_first=True
    )

    new_columns = [col for col in df.columns if col not in original_columns]
    return df, new_columns

In [ ]:
categorical_feats = [feat for feat in df.columns if len(df[feat].unique()) <= 10 and feat != "Outcome"]
categorical_feats

In [ ]:
df, new_cols_ohe = one_hot_encoder(df, categorical_feats)
new_cols_ohe

In [ ]:
df.head()

## Feature Scaling

In [ ]:
like_num = [col for col in df.columns if df[col].dtypes != 'O' and len(df[col].value_counts()) < 10]
no_need_to_scaled = new_cols_ohe + ["Outcome"] + like_num
cols_need_scale = [col for col in df.columns if col not in no_need_to_scaled]

print("List of columns that need to be scaled:\n", cols_need_scale)
rs = RobustScaler()
df.loc[:, cols_need_scale] = rs.fit_transform(df[cols_need_scale])
print("Feature Scaling, Done!")

In [ ]:
like_num

In [ ]:
df.head()

In [ ]:
df.info()

# 6.&nbsp;Modeling

## Notes: Metric Evaluation

See [classification_report](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html)

The choice between micro-average, macro-average, or weighted-average in the classification_report from scikit-learn depends on your specific use case and the characteristics of your data. Each average type provides a different perspective on the overall model performance.

Here's a brief explanation of each average type:

1. **Micro-average:**

    Calculates metrics globally by considering all instances together.

    Suitable when classes are imbalanced, and you want to treat all instances equally.
    Gives equal weight to each data point, regardless of class.

2. **Macro-average:**

    Calculates metrics for each class independently and then takes the unweighted average.

    Suitable when you want to evaluate the overall performance across all classes without considering class imbalances.
    Gives equal weight to each class, regardless of the number of instances in each class.

3. **Weighted-average:**

    Calculates metrics for each class independently and then takes the average, weighted by the number of true instances for each class.

    Suitable when classes are imbalanced, and you want to give more importance to the performance on larger classes.
    Provides a balanced view of the overall performance by accounting for class imbalances.

In summary:

- Use micro-average when you want to treat all instances equally, especially in the presence of class imbalances.

- Use macro-average when you want to evaluate the overall performance without considering class imbalances.

- Use weighted-average when you want to account for class imbalances and give more importance to the larger classes.


It's essential to choose the appropriate average based on the goals of your analysis and the nature of your data.

In some cases, you may need to consider multiple metrics and averages to get a comprehensive understanding of your model's performance.

In [ ]:
X = df.drop("Outcome", axis=1)
y = df["Outcome"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.1,    # 10% for testing
    stratify=y,       # Stratified sampling based on labels
    random_state=42   # Random seed for reproducibility
)

In [ ]:
print("y train:", y_train.value_counts(normalize=True))
print("\n")
print("y test:", y_test.value_counts(normalize=True))

In [ ]:
from sklearn.metrics import (
    accuracy_score, precision_score,
    recall_score, f1_score
)

In [ ]:
models = [
    ('LR', LogisticRegression()),
    ('KNN', KNeighborsClassifier()),
    ('CART', DecisionTreeClassifier()),
    ('RF', RandomForestClassifier()),
    ('SVC', SVC(gamma='auto')),
    ('XGBM', XGBClassifier()),
    ('GB', GradientBoostingClassifier()),
    ('LightGB', LGBMClassifier())
]

# Evaluate each model in turn
scorings = [
    'accuracy', 'f1_macro',
    'precision_macro', 'recall_macro'
]

model_perf = {}

# For each model
for name, model in models:

    results = {}

    # for each scorings
    for scoring in scorings:
        score_mean = []
        score_std = []

        # Define K-Fold
        kfold = KFold(
            n_splits=10, shuffle=True,
            random_state=42)

        # Training with cross validation
        cv_results = cross_val_score(
            model, X_train, y_train,
            cv=kfold, scoring=scoring)

        # Save Training Result
        results[scoring] = {
            'train_mean': cv_results.mean(),
            'train_std': cv_results.std()
        }

    model_perf[name] = results

In [ ]:
model_perf

In [ ]:
focus_metric = 'recall_macro'
perf_data = {
    "model_type": [],
    f"train_avg_{focus_metric}": [],
    f"train_stddev_{focus_metric}": []
}

for model_name, perf in model_perf.items():
    mean = perf[focus_metric]['train_mean']
    std = perf[focus_metric]['train_std']
    perf_data["model_type"].append(model_name)
    perf_data[f"train_avg_{focus_metric}"].append(mean)
    perf_data[f"train_stddev_{focus_metric}"].append(std)

eval_result = pd.DataFrame(perf_data)
eval_result

In [ ]:
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred_test = model.predict(X_test)

# Evaluate the model on the test set
accuracy_test = accuracy_score(y_test, y_pred_test)
precision_test = precision_score(y_test, y_pred_test, average='macro')
recall_test = recall_score(y_test, y_pred_test, average='macro')
f1_test = f1_score(y_test, y_pred_test, average='macro')

# Print or use the test set scores as needed
print(f"Test Set Accuracy: {accuracy_test:.4f}")
print(f"Test Set Precision: {precision_test:.4f}")
print(f"Test Set Recall: {recall_test:.4f}")
print(f"Test Set F1 Score: {f1_test:.4f}")

## 6.1 Model Hyper-Parameter Tuning

In [ ]:
# Let's choose the highest 4 models
# GBM
gbm_model = GradientBoostingClassifier()

# Model Tuning
gbm_params = {
    "learning_rate": [0.001, 0.01, 0.1],
    "max_depth": [3, 5, 8],
    "n_estimators": [200, 500, 1000],
    "subsample": [1, 0.5, 0.8]
}

gbm_cv_model = GridSearchCV(
    gbm_model,
    gbm_params,
    cv=3,
    n_jobs=-1,
    verbose=2).fit(X, y)

print(gbm_cv_model.best_params_)

# Final Model
gbm_tuned = GradientBoostingClassifier(**gbm_cv_model.best_params_).fit(X_train, y_train)

In [ ]:
# LightGBM:
lgb_model = LGBMClassifier()

# Model Tuning
lgbm_params = {
    "learning_rate": [0.01, 0.5, 1],
    "n_estimators": [200, 500, 1000],
    "max_depth": [6, 8, 10],
    "colsample_bytree": [1, 0.4, 0.5]
}

lgbm_cv_model = GridSearchCV(
    lgb_model,
    lgbm_params,
    cv=3,
    n_jobs=-1,
    verbose=2).fit(X, y)

print(lgbm_cv_model.best_params_)

# Final Model
lgbm_tuned = LGBMClassifier(**lgbm_cv_model.best_params_).fit(X_train, y_train)

In [ ]:
# Random Forests:
rf_model = RandomForestClassifier()

# Model Tuning
rf_params = {
    "n_estimators" :[200, 500, 1000],
    "max_features": [3, 5, 7],
    "min_samples_split": [2, 5, 10],
    "max_depth": [5, 8, None]
}

rf_cv_model = GridSearchCV(
    rf_model,
    rf_params,
    cv=3,
    n_jobs=-1,
    verbose=2).fit(X_train, y_train)

print(rf_cv_model.best_params_)

# Final Model
rf_tuned = RandomForestClassifier(**rf_cv_model.best_params_).fit(X, y)

In [ ]:
# XGB
xgb_model = XGBClassifier()

# Model Tuning
xgb_params = {
    "learning_rate": [0.01, 0.1, 0.2],
    "min_samples_split": np.linspace(0.1, 0.5, 10),
    "max_depth":[3, 5, 8],
    "subsample":[0.5, 0.9, 1.0],
    "n_estimators": [100, 1000]
}

xgb_cv_model = GridSearchCV(
    xgb_model,
    xgb_params,
    cv=3,
    n_jobs=-1,
    verbose=2).fit(X_train, y_train)

print(xgb_cv_model.best_params_)

xgb_tuned = XGBClassifier(**xgb_cv_model.best_params_).fit(X, y)

In [ ]:
# evaluate each model in turn
models = [
    ('RF', rf_tuned),
    ('GBM', gbm_tuned),
    ("LightGBM", lgbm_tuned),
    ("XGB", xgb_tuned),
    #...,
    #...,
]

# Evaluate each model in turn
scorings = [
    'accuracy', 'f1_macro',
    'precision_macro', 'recall_macro'
]

model_training_perf = {}

# For each model
for name, model in models:

    results = {}

    # for each scorings
    for scoring in scorings:
        score_mean = []
        score_std = []

        # Define K-Fold
        kfold = KFold(
            n_splits=5, shuffle=True,
            random_state=42)

        # Training with cross validation
        cv_results = cross_val_score(
            model, X_test, y_test,
            cv=kfold, scoring=scoring)

        # Save Training Result
        results[scoring] = {
            'train_mean': cv_results.mean(),
            'train_std': cv_results.std()
        }

    model_training_perf[name] = results

## 6.2 Model Training Evaluation

In [ ]:
import pickle
import os
os.makedirs("models")

# Define the list of models with their names
models = [
    ('RF', rf_tuned),
    ('GBM', gbm_tuned),
    ("LightGBM", lgbm_tuned),
    ("XGB", xgb_tuned),
]

# Iterate over each model in the list
for model_name, model in models:
    # Specify the file path where you want to save the model
    file_path = f"models/{model_name}_model.pkl"

    # Open the file in binary write mode
    with open(file_path, 'wb') as file:
        # Use pickle.dump() to serialize and save the model to the file
        pickle.dump(model, file)

In [ ]:
# model_training_perf

In [ ]:
file_path = "model_training_perf.pkl"
with open(file_path, 'wb') as file:
    pickle.dump(model_training_perf, file)

In [ ]:
model_training_perf_df = pd.concat(
    {
        k: pd.DataFrame.from_dict(v, 'index') for k, v in model_training_perf.items()
    },
    axis=0
)

model_training_perf_df.index.rename(
    ['model_name', 'eval_metric'],
    inplace=True
)

model_training_perf_df

In [ ]:
import seaborn as sns
sns.set_theme(style="whitegrid")

In [ ]:
train_viz_data = (
    model_training_perf_df
    .loc[(slice(None), slice(None)), :].
    train_mean
    .reset_index()
)

train_viz_data

In [ ]:
# fig, ax = plt.subplots()
g=sns.catplot(
    data=train_viz_data,
    kind="bar",
    x="model_name",
    y="train_mean",
    hue="eval_metric",
    palette="dark",
    alpha=.6,
    height=6
);

# Set axis labels
g.set_axis_labels("", "mean_score")

# Set legend title
g.legend.set_title("Metric")

# Set figure title
g.fig.suptitle("Model Evaluation On Train Data\n")

# Set y-ticks with a scale of 0.05
g.ax.set_yticks([i * 0.05 for i in range(int(g.ax.get_ylim()[1] / 0.05) + 1)])

# Show the plot
plt.show()

## 6.3 Model Selection
Evaluation on Test Data

In [ ]:
import pickle

# Define a list to store the loaded models
loaded_models = []

# Define the list of model names
model_names = ['RF', 'GBM', 'LightGBM', 'XGB']

# Iterate over each model name
for model_name in model_names:
    # Specify the file path of the corresponding .pkl file
    file_path = f"models/{model_name}_model.pkl"

    # Open the file in binary read mode
    with open(file_path, 'rb') as file:
        # Use pickle.load() to deserialize and load the model from the file
        loaded_model = pickle.load(file)

        # Append the loaded model to the list of loaded models
        loaded_models.append((model_name, loaded_model))

In [ ]:
from sklearn.metrics import (
    accuracy_score, f1_score,
    precision_score, recall_score
)

# Define lists to store the evaluation metrics for each model
accuracies = []
f1_scores = []
precisions = []
recalls = []

# Iterate over each loaded model
for model_name, loaded_model in loaded_models:
    # Predict using the loaded model on the test data
    y_pred = loaded_model.predict(X_test)

    # Calculate evaluation metrics
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)

    # Append the metrics to the respective lists
    accuracies.append((model_name, accuracy))
    f1_scores.append((model_name, f1))
    precisions.append((model_name, precision))
    recalls.append((model_name, recall))

evaluation_results = pd.DataFrame({
    'Model': [model_name for model_name, _ in loaded_models],
    'Accuracy': [accuracy for _, accuracy in accuracies],
    'F1 Score': [f1 for _, f1 in f1_scores],
    'Precision': [precision for _, precision in precisions],
    'Recall': [recall for _, recall in recalls],
})

# Print the DataFrame
evaluation_results

In [ ]:
# Melt the evaluation_results DataFrame
melted_evaluation_results = pd.melt(
    evaluation_results,
    id_vars=['Model'],
    value_vars=[
        'Accuracy', 'F1 Score',
        'Precision', 'Recall'
    ],
    var_name='Metric',
    value_name='Value'
)

# Print the melted DataFrame
melted_evaluation_results

In [ ]:
# fig, ax = plt.subplots()
g=sns.catplot(
    data=melted_evaluation_results,
    kind="bar",
    x="Model",
    y="Value",
    hue="Metric",
    palette="dark",
    alpha=.6,
    height=6
);

# Set axis labels
g.set_axis_labels("", "score")

# Set legend title
g.legend.set_title("Metric")

# Set figure title
g.fig.suptitle("Model Evaluation On Test Data")

# Set y-ticks with a scale of 0.05
g.ax.set_yticks([i * 0.05 for i in range(int(g.ax.get_ylim()[1] / 0.05) + 1)])

# Show the plot
plt.show()